## Deliverable 2. Create a Customer Travel Destinations Map.

In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavieng,PG,-2.5744,150.7967,83.44,73,60,12.55,broken clouds
1,1,Provideniya,RU,64.3833,-173.3000,48.36,81,100,13.42,overcast clouds
2,2,Himare,AL,40.1017,19.7447,74.08,69,0,3.02,clear sky
3,3,Moose Factory,CA,51.2585,-80.6094,69.80,64,0,8.05,clear sky
4,4,Kununurra,AU,-15.7667,128.7333,67.98,94,2,0.00,clear sky


In [26]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavieng,PG,-2.5744,150.7967,83.44,73,60,12.55,broken clouds
2,2,Himare,AL,40.1017,19.7447,74.08,69,0,3.02,clear sky
5,5,Georgetown,MY,5.4112,100.3354,78.73,95,20,2.30,few clouds
11,11,Atuona,PF,-9.8000,-139.0333,76.80,81,29,28.16,light rain
14,14,Inhambane,MZ,-23.8650,35.3833,70.52,63,100,7.49,overcast clouds
21,21,Murakami,JP,38.2333,139.4833,76.89,89,100,2.28,moderate rain
29,29,Fortuna,US,40.5982,-124.1573,75.65,45,89,6.96,overcast clouds
30,30,Itoman,JP,26.1247,127.6694,80.62,78,40,16.11,scattered clouds
31,31,Butaritari,KI,3.0707,172.7902,81.72,72,100,14.70,overcast clouds
34,34,Kampot,KH,10.6167,104.1833,75.25,94,100,8.59,overcast clouds


In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavieng,PG,83.44,broken clouds,-2.5744,150.7967,
2,Himare,AL,74.08,clear sky,40.1017,19.7447,
5,Georgetown,MY,78.73,few clouds,5.4112,100.3354,
11,Atuona,PF,76.80,light rain,-9.8000,-139.0333,
14,Inhambane,MZ,70.52,overcast clouds,-23.8650,35.3833,
21,Murakami,JP,76.89,moderate rain,38.2333,139.4833,
29,Fortuna,US,75.65,overcast clouds,40.5982,-124.1573,
30,Itoman,JP,80.62,scattered clouds,26.1247,127.6694,
31,Butaritari,KI,81.72,overcast clouds,3.0707,172.7902,
34,Kampot,KH,75.25,overcast clouds,10.6167,104.1833,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavieng,PG,83.44,broken clouds,-2.5744,150.7967,Nusa Island Retreat
2,Himare,AL,74.08,clear sky,40.1017,19.7447,Hotel Dima
5,Georgetown,MY,78.73,few clouds,5.4112,100.3354,Cititel Penang
11,Atuona,PF,76.80,light rain,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
14,Inhambane,MZ,70.52,overcast clouds,-23.8650,35.3833,Oceano


In [33]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))